<a href="https://colab.research.google.com/github/bbaronSVK/pt/blob/main/summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author:** Dr. Shahriar Hossain <br>
**Topic of the code:** Long Text Summarization with Hugging Face Transformers <br>
**Video explaining this code:** https://youtu.be/UTlb5C7xG9A <br>
**My YT Channel:** https://www.youtube.com/@C4A <br>
**Web:** https://computing4all.com/ <br>

In [ ]:
# Install the transformers package from Hugging Face:
!pip install transformers
!pip install torch
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.1 MB/s eta 0:00:00


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(
    'facebook/bart-large-cnn')

In [ ]:
def summarize(text, maxSummarylength=500):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=maxSummarylength,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
def split_text_into_pieces(text,
                           max_tokens=900,
                           overlapPercent=10):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    # Convert the token pieces back into text
    text_pieces = [tokenizer.decode(
        tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]

    return text_pieces


In [ ]:

def recursive_summarize(text, max_length=200, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length/3*2)
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [ ]:
# Example usage
text = '''Dlouho se tvářila, že je spokojená a v jejím životě je všechno v pořádku, ve skutečnosti si ale nechtěla přiznat, že si toho na sebe nakládala až příliš. „Je to dvanáct let, co jsem se dohnala až na psychiatrii,“ svěřila se Šípková v pořadu Na kafeečko s tím, že ihned po první návštěvě u specialisty vyfasovala antidepresiva, která pak užívala pět let.

Těžká nespavost
Varovné signály, jež Šípkové hlásily, že je čas udělat razantní změnu, se hlásily každou noc. Bývalá šampionka ve sportovním aerobiku a později fitness trenérka, televizní moderátorka a propagátorka aktivního života totiž začala trpět těžkou nespavostí. „S tou se opravdu nedá žít, nemáte energii na nic,“ vysvětlila a dodala, že se jeden čas doslova bála své vlastní postele. „Nemohla jsem spát v jedné místnosti, musela jsem se pořád stěhovat,“ přiznala v rozhovoru na iDNES.cz.

Poslední rána
Když to už už vypadalo, že konečně přeskládala život k lepšímu, přišla další rána: Během jedné z lekcí, které vedla, začala zničehonic zvracet. „Lekci jsem dokončila, protože já jsem přece nejlepší! Kamarádky mě odvezly z tělocvičny domů. Můj současný muž se zajímal, co se děje, a poslal za mnou dceru,“ zavzpomínala Šípková na situaci, jež jí pravděpodobně zachránila život. Společně mu pak zavolaly. „Vzal si mě k telefonu a já začala žvatlat nesmysly. Nečekal a poslal za mnou rychlou. To už jsem měla povislý koutek úst, šla jsem rovnou na JIP. A tam se zjistilo masivní krvácení do mozku!“ popsala chvíle, kdy jí šlo bez přehánění o život. „Doktor mi řekl, že mě od smrti dělily dva centimetry. Kdyby se totiž krev dostala o ten kousíček dál, už bych si na nedostatek odpočinku opravdu nestěžovala…“ dodala Šípková, kterou mozková příhoda na půl roku zcela vyřadila z provozu.

Nové tělo
Po cévní mozkové příhodě musela Šípková změnit nejen životní styl, ale i myšlení. Dostala na čas zákaz sportovat a do toho se ohlásila menopauza. Tělo celoživotní sportovkyně se tak téměř ze dne na den začalo měnit. „Za měsíc jsem měla šest kilo nahoře. Najednou jsem měla velká prsa, břicho a viděla, jak jdu do podoby své maminky,“ přiznala Olga, jež se pak dlouho učila mít své nové tělo ráda.

'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  7
****************************************************
Piece: 1  out of  7 pieces
Dlouho se tvářila, že je spokojená a v jejím životě je všechno v pořádku, ve skutečnosti si ale nechtěla přiznat, že si toho na sebe nakládala až příliš. „Je to dvanáct let, co jsem se dohnala až na psychiatrii,“ svěřila se Šípková v pořadu Na kafeečko s tím, že ihned po první návštěvě u specialisty vyfasovala antidepresiva, která pak užívala pět let.

 

SUMNMARY:  Dlouho se tvářila, že je spokojená a v jejím životě je všechno v pořádku. ‘Je to dvanáct let, co jsem se dohnala až na psychiatrii,’ svěřil se Šípková.
****************************************************
****************************************************
Piece: 2  out of  7 pieces
, která pak užívala pět let.

Těžká nespavost
Varovné signály, jež Šípkové hlásily, že je čas udělat razantní změnu, se hlásily každou noc. Bývalá šampionka ve sportovním aerobiku a později fitness tren

KeyboardInterrupt: ignored